# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [4]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [5]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [8]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the page.
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height
    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [9]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [10]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 富邦金1月賺95億、每股賺近1元　雙創歷年元月新高

▲富邦金控今天公布1月自結稅後盈餘95.14億元，每股盈餘0.93元，雙創歷年1月新高。（圖／資料照）
記者紀佳妘／台北報導
富邦金控（2881）今（10）日公布1月自結稅後淨利95.14億元，相較去年同期成長158%，每股盈餘（EPS）為0.93元，稅後淨利和EPS雙雙創下歷年1月新高，穩居金控獲利王寶座。
富邦金控旗下子公司獲利均較去年同期成長，其中又以富邦人壽的投資收益提升影響最大，1月稅後淨利為59.85億元，較去年同期大幅成長，創下2016年來1月獲利新高，受惠於投資績效表現亮眼，1月初年度保費收入（FYP）達147億元、總保費收入為541億元。
台北富邦銀行1月稅後淨利23.26億元，主要是因淨利息收益成長動能穩健。北富銀資產品質穩健，1月底逾放比率為0.18%，備抵呆帳覆蓋率為735%。
至於其他子公司獲利表現，富邦產險1月稅後淨利7.07億元，較去年同期大幅成長121%，主要受惠於優異的投資績效和承保業務表現；富邦證券1月稅後淨利1.17億元，獲利動能較去年同期減緩，主要受1月交易天數較少及部位收益減少的影響。
----------------------------------------------------------------------

[1] 國泰金投資表現亮眼　1月大賺101億首破百億元大關

▲國泰金今天公布1月稅後盈餘為101.3億元，每股稅後盈餘達0.76元。（圖／資料照）
記者紀佳妘／台北報導
國泰金今（10）日公布1月自結獲利數字，1月稅前淨利116.2億元，稅後淨利101.3億元，年增115.5%，每股稅後盈餘（EPS）達0.76元，今年首月獲利率先突破百億元大關。
國泰金旗下子公司國泰人壽持續強化價值導向策略，國壽1月稅後淨利66.1億元，主要是1月投資績效良好，業務面持續價值導向策略，保單銷售以傳統型分期繳及投資型保單為主，1月初年度保費收入（FYP）金額為198億元、初年度等價保費（FYPE）金額為110億元，皆為業界第一。
國泰世華銀行1月稅後純益31.9億元，每股盈餘為0.31元。國泰金表示，國泰世華獲利表

2018年下半年開啟了美中貿易戰，加上工資的上漲，在中國的台商立即面臨「go south or go home（南向或返台）」的抉擇，其中，高附加價值、可自動化與中大型資本規模的企業形成回台投資的風潮。
根據官方的統計資料顯示，截至2019年11月底為止，已經核准之回台投資金額超過8,000億元，光是和碩集團的和碩（4938）、景碩（3189）與晶碩（6491）3家公司的回台投資金額就超過330億元。
然而，台商回流所面臨的第一個難題就是缺地，更精準的說法是缺少「合適的」土地與廠房。而合適的工業區地價高漲，正反映了缺地情況的嚴重性。從2008年至2018年，中北部熱門的工業區地價，至少都翻倍。根據國際房地產服務商第一太平戴維斯的統計，漲幅最高的龜山工業區達314%，居次的中壢工業區為300%；中部以台中工業區的286%居冠，其次是彰濱工業區的167%。
另外，商仲世邦魏理仕的資料顯示，2018年工業地產交易量達788億元，年增50%，創十年來新高，其中，廠房與廠辦的交易量逾550億元，年增更達77%，顯見貿易戰所引發的工業資產需求已然展開。根據調查，2020年新增產業用地將達384公頃，明顯超過2019年的282公頃；至2021年新增標準廠房面積將達3萬2,823坪，更遠超過2019年的7,565坪。
新的廠辦或工業土地開發案可以享有某些獎勵措施，例如：提高容積率、地價稅與房屋稅之補貼，擁有優質工業資產自行開發、受委託開發的企業將獲得激勵，使其成為這2年～3年極佳的投資趨勢，而且此趨勢不會明顯受到短期景氣面與股市波動的影響，會是中長期投資者的優先選擇。
工業資產開發的一個成功範例是德昌（5511），公司2009年4月取得「大埔美精密機械園區」開發案，2011年與2012年陸續有入帳貢獻。每股盈餘（EPS）自2009年的1.96元成長至2012年的5.93元，同時間的股價則從約11元漲至38元。觀察德昌的經驗可知，股價會依開發進度分段上漲，而開發的價值貢獻長達數年。
筆者在觀察此一趨勢並且參考德昌後，列出幾項選股準則：一是低股價、低位階，成長性與股價爆發力較佳；二是適中的股本，例如：小於30億元，EPS貢獻度才明顯；三是本業最好有獲利，即使處於虧損也能在可見的近期轉盈；四是淨值至少接近票面，而且股價淨值比低於1倍，代表即使有累計虧損亦不算嚴重；最後也是最重要的是，開發

►只顧著滑手機？臉也要天然發光
----------------------------------------------------------------------

[17] 疫情擴大復工困難！伺服器提前備貨　短期不至於斷料

▲武漢肺炎疫情持續發燒，台商在大陸廠區復工日期未定，蘋果供應鏈有斷鏈的疑慮。（圖／記者湯興漢攝）
記者周康玉／台北報導
科技廠因受武漢疫情影響，在中國大陸廠區延後至今日開工，更因為部分地區下達封城令，導致台廠部分廠區復工困難。然而，研調機構指出，受惠於農曆年前的預先備料與部分提前復工，中國伺服器出貨暫時不受疫情影響，短期不至於面臨斷料。
TrendForce記憶體儲存研究（DRAMeXchange）指出，中國伺服器廠商除直接人員預定在2月10日陸續復工外，其餘間接人員皆於2月3日恢復上班，而部份廠商因取得政府許可於2月3日即復工，恢復狀況優於預期。
以往春節期間，廠商大多會提前一個月增加產量並備妥物料，以利年節後順利出貨，故儘管復工向後遞延，各大ODM廠均認為短期影響甚微；倘若2/10復工狀況欠佳，後續會進行增產。
TrendForce分析師劉家豪表示，以出貨訂單的規劃來看，因代工廠產線部分已轉移至台灣等第三地生產，對於美系資料中心業者的影響甚微，而浪潮、華為等中國廠商訂單則視疫情的控制狀況而定，短期內衝擊有限。整體而言，目前評估伺服器整機的全年出貨不會有太大波動。
劉家豪表示，最大的隱憂是上游的板材原料（PCB），因勞力密集度較高，返工狀況對產能的影響仍須觀察。然而伺服器業者均有先備庫存，短期不至於面臨斷料。
從應用領域來看，中國大陸今年訂單主要仍來自電信業者與網路資料中心。其中，電信業者的5G相關標案未因疫情而出現砍單的狀況，僅部分向後遞延。其中，中國5G相關伺服器的主要供應商為華為與中興，兩者均已提前申請復工，中國電信商今年的需求均可如期落實；同時，預估需求規模將較去年成長120%，達到30萬台。
而中國資料中心方面，部分廠商如騰訊與字節跳動則因疫情受惠。其中，騰訊得利於諸多學校與公司實行遠距教學、辦公，今年伺服器訂單較去年成長兩成，來到33萬台左右；字節跳動則因中國國內短視頻流量增加且海外市場拓展順利，今年伺服器需求預估將超越阿里雲與百度，達到25萬台左右。
---------------------------------

▲武漢肺炎疫情持續發燒，不少台商大陸廠區難以全部復工，台積電南京廠及上海廠則是正常開工。（圖／記者湯興漢攝）
記者周康玉／台北報導
台積電（2330）今（10）日公布1月營收為1036.83億元，月增0.4%，較去年同期增加32.8%，連六月守在千億元大關，為歷年同期新高。
台積電今日以321.5元開出，開低後逐步回神，以327.5元作收；台積電ADR上周五收56.20美元，下跌1.82美元或3.14%，ADR收盤價折合新台幣每股約338.61元。
台積電第4季合併營收達3172.4億元，創單季營收歷史新高，以美金計算，營收為103.9億元，年增10.6%、季增10.6%；第4季毛利率為50.2%，營業利益率為39.2%，稅後純益率36.6%，皆較上季與前年同期增長，並超越財測。全年營收達1兆699.85億元，年增3.7%，連續2年破兆元大關，再創營收新高。
受到武漢肺炎疫情影響，中國大陸政府先是宣布延長春節假期至2月2日，2月3日起正常上班，爾後又緊急宣布，部分地區延後至2月10日才能開工。台積電在中國大陸廠區亦受影響，台積電在南京設有12吋晶圓廠南京廠區原本於2月3日開工，以及上海為8吋晶圓廠則2月10日開工。然而南京於4日宣布全面實施小區封閉式管理，台積電配合當地政府調整把原定2月3日全面復工延至10日，今日兩廠區順利開工。
台積電南京廠目前月產能1.5萬片，占台積電總產能僅約1%-3%；台積電上月法說會表示，今年底月產能目標達2萬片目標不變。
台積電第4季獲利優於預期，且今（2020）年第1季營收展望淡季不淡；今年資本支出增加至150~160億美元，和去年150億元同樣維持在高檔，顯現在其製程具領導地位，以及掌握未來數年5G及HPC領域成長契機下，對於前景看好。
台積電於上月中旬法說會後，外資全面上調目標價，花旗證目標價402元，高盛390元等，法人指出，外資目標價平均值是386元，離目前股價約20%空間。
近期受到武漢疫情衝擊，台積電股價一度重挫至315元，跌破季線321元，然而近日逐步回溫，近期處在季線保衛戰。法人表示，等待疫情系統性風險降低，買盤則會陸續回流，然而一旦失守季線，台股指數必破年線。
----------------------------------------------------------------------

[31] 疫情

記者吳靜君／台北報導
武漢病毒疫情延燒，台酒表示，屏東酒廠已經開始生產75度酒精，今（10）日起在屏東、高雄等地開始銷售，明日開始，全國都可以買到75度酒精，包含台北等地，此外台酒表示，花蓮酒廠會加入生產75度酒精的行列，所以75度酒精與95度酒精，共計28萬3200瓶，備貨充足，足夠讓民眾購買，也請民眾切勿囤貨。
▲武漢肺炎疫情延燒，台酒火力全開，力求每日供貨28萬瓶酒精以上，以滿足民眾的需求。（圖／記者李毓康攝）
隨著疫情的延燒，除了口罩之外，防疫必備品的酒精也成為民眾搶購的用品之一。台酒表示，目前在全國的營業所、台酒便利商店以及展售中心共計120處銷售6600打95度的酒精。
另外，今日開始屏東酒廠已經加入生產75度的酒精，以600ml計算的話，每日可以生產4000打提供給民眾購買，首先以屏東、高雄開賣，台北等其他等地因為運送的關係，則要等到明日才可以買得到，而為了避免民眾的恐慌心理，600ml酒精售價只有75元。花蓮酒廠也會加入生產300的75度酒精，預計生產1萬3000打，提供民眾防疫用。
所以明日開始，台酒火力全開，每日可供應28萬3200多瓶95度以及75度酒精，提供民眾購買，也就是說，扣除每日約8萬瓶的95度酒精外，明日起75度酒精的供貨量會超過20萬瓶。台酒表示，現在也會視民眾購買的情形，適時的增開台中酒廠、宜蘭酒廠來生產酒精，因此民眾不用擔心，也視需求購買切勿囤貨。
----------------------------------------------------------------------

[42] 武漢肺炎疫情蔓延　蘋果大陸門市停業時間再延長至2月15日

▲蘋果決定延長中國大陸門市停業時間 。（圖／路透社）
記者陳心怡／外電報導
武漢肺炎疫情持續延燒，根據《路透社》報導，蘋果決定延長中國大陸門市停業時間至2月15日。
蘋果上週曾表示，基於謹慎考慮以及尋求專業醫療專家建議後，決定暫停在中國大陸42家門市的營業，直到2月10日。不過蘋果日前在中國大陸網站顯示，將延長在中國大部份門市停業時間至2月15日。
蘋果再度延長中國大陸門市停業時間，可能意味著蘋果在中國市場的銷售情況，受疫情影響的時間比原本預估的還來得長，這將會大幅影響蘋果的財報及營收獲利表現。
---------------------------------------

▲美股道瓊下跌 277.26 點。回頭看台指期今（10）日開盤下跌 134 點，來到 11,455 點。（圖／路透）
記者陳心怡／台北報導
美國 1 月非農就業報告好壞參半，加上中國大陸連封 4 省防疫，讓華爾街對中國大陸經濟產生新的擔憂，週五（7日）美股四大指數全下跌。回頭看台指期今（10）日開盤下跌 134 點，來到 11,455 點。
道瓊指數下跌 277.26 點或 0.94%，收 29102.51 點；那斯達克指數下跌 51.64 點或 0.54%，收 9520.51 點；標普 500 指數下跌 18.07 點或 0.54%，收 3327.71 點；費城半導體指數下跌 45.28 點或 2.37%，收 1864.36 點。
中國各省預計今起將陸續復工，但陸續有供應鏈宣布進一步延後開工，市場擔憂受疫情衝擊2、3 月業績，法人表示需持續關注中國大陸復工情況，短線慎防大幅震盪整理。
截至今早8：47，台指期下跌 154 點，來到 11,435 點。
----------------------------------------------------------------------

[56] 「富士康復工被政府叫停」為不實訊息　深圳市政府單位為鴻海闢謠

▲鴻海旗下富士康。（圖／路透）
記者姚惠茹／台北報導
新冠肺炎疫情持續延燒，鴻海（2317）明（10）日是否復工受到各界關注，但是網傳「富士康恢復生產的計劃被政府叫停」，深圳市龍華區工業和信息化局今（9）日特別澄清為不實訊息，並強調正在審核富士康是否符合復工的「八個必須」情況。
深圳市龍華區工業和信息化局發布的內容表示，根據《廣東省疫情防控指揮部辦公室關於實施企業提前復工報備的通知》《深圳市新型冠狀病毒感染的肺炎疫情防控指揮部辦公室關於實施企業復產復工報備制度的通知》及補充通知要求，富士康集團旗下3家法人企業2月6日提出企業復產復工備案表及疫情防控承諾書。
龍華區工業和信息化局已聯合各部門組成現場核查小組，正對富士康集團法人企業進行核查，重點核查企業是否落實復產復工「八個必須」情況：一、必須建立一套企業防控管理體系；二、必須建立一本企業防控工作台帳；三、必須組織復工員工一日測量兩次體溫；四、必須每日為每位員工準備至少一個口罩；五、必須落實一處符合疫情防控要求的隔離場所；六、必須每天進行至少一次衛生消毒；

▲瞄準民眾需求，各大銀行均推出高利美元定存方案，其中利率最高喊出6%。（示意圖／路透）
記者紀佳妘／台北報導
美元定存是許多投資人喜愛的外幣投資選項，各大銀行均推出高利美元定存方案，包括花旗銀、星展銀、渣打銀、中信銀等4家銀，都有推出短天期的美元優利定存專案，其中利率最高喊出6%，吸引投資人的目光。
看準客戶需求，國銀、外銀均推出美元優利定存方案，其中目前利率最高的是花旗銀行，到3月底止推出「短天期」美元限時高利活動，7天期定存年利率高達6%、14天期也有5%的年利率。不過，資金的條件要求是以「以非美金之資金轉換為美金後辦理定存」，且最低承作金額為3萬美元。
星展銀也推出「iWealth美元高利定存專案」，從即日起到3月31日為止，首筆定存限定最低承作金額為2萬美元，可享1個月期、優惠年利率3.36%，不過僅限新星展豐盛理財客戶及星展豐盛私人客戶，且不得超過5萬美元。
針對偏好外幣資產配置的消費者，渣打銀行提供美元優利定存專案，單筆存入達2萬美元，可享2個月期年息2.6%；若存滿6萬美元，則可享2個月期年息則突破3字頭，來到3.05%，不同的優惠高利方案，有利消費者進行多元均衡的資產配置規劃。
至於民營銀行部分，中信銀提供多種美元優利定存專案，到2月27日止，無論新舊資金，客戶只需依牌告匯率換匯轉存，單筆5,000美元（含）以上即可享14天期定存年利率3%、1個月期定存年利率2.28%。此外，為滿足客戶靈活運用的需求，特別推出3個月期定存年利率2.08%。
----------------------------------------------------------------------

[69] 將房子送下一代　國稅局解惑：要考慮贈與稅和房地合一稅

▲將不動產轉移給下一代，若要出售要考慮取得成本的問題。（圖／翻攝自pixabay）
記者吳靜君／台北報導
如果名下有不動產，想要移轉給下一代，是賣給下一代好？還是贈與給下一代好呢？財政部南區國稅局解惑，近期有民眾詢問，其名下一筆不動產，公告及評定現值為200萬元，市值大約1000萬元，要何種方式比較節稅？不同的移轉方式會不會影響兒子出售該筆不動產。
南區國稅局表示，不動產移轉主要有兩種方式，有買賣與贈與兩種方式，其中贈與方式可能會須繳納贈與稅；另外因房地合一所得稅已在2016年開始實施，如果該民眾的兒子有要

▲南屯精科好宅開放參觀，吸引逾百人前來參觀。（圖／台中市政府提供，下同）
記者鄧木卿／台中報導
台中市首座民間回饋社會住宅「南屯區精科樂活好宅」，預計6月開放承租入住，市府住宅發展工程處原訂14日截止受理申請入住，考量武漢疫情持續延燒，主動延長申請期限至24日，希望分散現場申請人流，也讓有意願申請的市民能多點時間準備。
▲社會住宅「南屯區精科樂活好宅」預計6月開放入住。
住宅處表示，南屯精科樂活好宅位於文山三街366號，全棟地下3層、地上14層共190戶，其中1房型（13-19坪）120戶、2房型（21-31坪）44戶、3房型（35-36坪）26戶，每戶都有冷氣、廚具、衛浴設備、單人或雙人床箱、衣櫃、窗簾及燈具等，並設有托嬰中心、社會服務站等公共設施。
租金價格經3家不動產估價師評估後，再以7折以下招租，第1年更以市價5折租金優惠，1房型含管理費5900至8700元，2房型7900至1萬1600元，3房型1萬3100至1萬3500元。
住宅處指出，南屯精科樂活好宅與太平育賢社宅公告招租後，民眾反應熱烈，諮詢電話不斷，許多人表示希望進一步了解室內格局及設施設備，因此市府特別開放參觀，現場依報名順序安排看屋。
住宅處表示，南屯區公所及太平區公所所設置的臨時櫃檯，收件至14日，15日至24日的招租延長受理時間，民眾可到台中州廳辦公室現場申請，此外，今年首辦社會住宅線上申請，申請人可至住宅處網站（https://thd.taichung.gov.tw/）以線上填寫申請資料，並上傳相關文件，應檢附之財稅資料則可利用自然人憑證或健保卡至財政部電子稅務文件入口網(https://etd.etax.nat.gov.tw/apply-services/)線上申辦。
►我們的美麗就用白紙黑字寫下來
----------------------------------------------------------------------

[79] Google與Facebook放棄香港　重啟台灣、菲律賓海底電纜

▲GOOGLE。（圖／路透社）
記者余弦妙／綜合報導
Google和Facebook 原定建設香港至美國的跨太平洋海底電纜 PLCN（ Pacific Light Cable Network ），但至今工程已完成仍未獲批文。據了解，是因美國憂慮國家安全問題，聯邦通信委員會（

▲因為新型冠狀病毒疫情影響，愛立信也宣布退出MWC 2020。（圖／《ETtoday新聞雲》資料照）
記者邱倢芯／台北報導
繼先前LG因為新型冠狀病毒疫情而宣布退出今年的世界通訊大會（MWC）之後，通訊設備商愛立信（Ericsson）也決議退出MWC 2020。
愛立信指出，該公司視員工、客戶和其他利益關係人的健康與安全為公司的首要責任；由於新型冠狀病毒的爆發，將無法確保參展員工和客戶的健康與安全，已決定退出今年度的產業盛事MWC 2020。而原先在MWC當中愛立信將展出的內容，也暫時無法亮相，將會改以在各地區單獨舉辦「Ericsson Unboxed」活動。
事實上，無論是LG還是愛立信的退出，對於今年的MWC展會來說都是一大打擊，因為二家廠商都是屬於大型業者，這也將讓MWC 2020的舉行與否再添變數。
----------------------------------------------------------------------

[93] 最後一天！春節大紅包「7700萬」一次發完　大樂透3.6億號碼開出

▲大樂透春節大紅包77組100萬元今晚將一次送出，想中大獎的民眾可把握最後一次機會。（圖／記者許力方攝）
記者趙蔡州／綜合報導
大樂透自小年夜起連開11天，連槓7期頭獎上看3.6億元；春節大紅包則尚有77組未開出，7日將是最後一次加開，民眾不妨在最後一次拚拚看手氣，只要9個號碼對中6個號碼，總獎金7700萬元將由中獎組數均分
台彩7日晚間開出109000019期大樂透，獎號分別為05、15、19、22、30、42，特別號31。
春節大紅包加開獎碼為01、03、07、09、23、30、32、37、44，只要對中任6個獎號，剩餘77組總獎金7700萬元將由中獎組數均分。
----------------------------------------------------------------------

[94] 富士康致信深圳廠員工10日暫不回廠　鴻海聲明：配合做好安全復工準備

▲富士康。（圖／路透）
記者姚惠茹／台北報導
鴻海（2317）今（7）日針對外媒報導「致信深圳廠員工10日暫不回廠」一事，發出三點聲明表示，集團早已成立防疫指揮中心，時刻密切關注疫情發展態勢，並在當地政府指導下做好安全復工準備，更不針對單一園區產能排配及客戶產